In [11]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon, LineString
from shapely.prepared import prep
import matplotlib.pyplot as plt

# Load the Philippine waters shapefile
phil_waters = gpd.read_file("/home/cs-iesm-geostorm/Data/gis/vector/boundaries/waters/philippine_waters_filled.shp")



In [12]:
ph_shape = phil_waters.geometry.iloc[0]
ph_buffered = ph_shape.buffer(0.5)  # 0.5 degrees (since CRS is EPSG:4326)

In [13]:
par_coords = [
    (120, 25),
    (135, 25),
    (135, 5),
    (115, 5),
    (115, 15),
    (120, 21),
    (120, 25)  # Close the polygon
]

# Create PAR polygon
par_poly = Polygon(par_coords)
par_gdf = gpd.GeoDataFrame(geometry=[par_poly], crs="EPSG:4326")

In [14]:
# Load the IBTrACS data CSV
ibtracs = pd.read_csv("/home/cs-iesm-geostorm/Data/gis/vector/ibtracs/ibtracs.WP.list.v04r01.csv", low_memory=False ,skiprows = [1])
ibtracs = ibtracs[ibtracs.SEASON>=1979]
# Step 2: Get unique storm IDs
unique_sids = ibtracs["SID"].unique()

In [15]:
len(unique_sids)

1507

In [16]:
# Convert to GeoDataFrame
gdf_tracks = gpd.GeoDataFrame(
    ibtracs,
    geometry=gpd.points_from_xy(ibtracs["LON"], ibtracs["LAT"]),
    crs="EPSG:4326"
)

In [17]:
from shapely.geometry import LineString
from shapely.prepared import prep

# Prepare the PH and PAR polygons
ph_prepared = prep(ph_buffered)
par_prepared = prep(par_poly)

# Also get their bounding boxes for a fast first filter
ph_bounds = ph_buffered.bounds
par_bounds = par_poly.bounds

intersect_results = []

for sid, group in gdf_tracks.groupby("SID"):
    points = list(group.sort_values("ISO_TIME")["geometry"])
    if len(points) < 2:
        continue  # Skip storms with insufficient points

    track_line = LineString(points)
    line_bounds = track_line.bounds

    # Fast bounding box skip
    skip_ph = (line_bounds[2] < ph_bounds[0]) or (line_bounds[0] > ph_bounds[2]) or \
              (line_bounds[3] < ph_bounds[1]) or (line_bounds[1] > ph_bounds[3])

    skip_par = (line_bounds[2] < par_bounds[0]) or (line_bounds[0] > par_bounds[2]) or \
               (line_bounds[3] < par_bounds[1]) or (line_bounds[1] > par_bounds[3])

    # Use prepped geometry for faster intersects
    intersects_ph = False if skip_ph else ph_prepared.intersects(track_line)
    intersects_par = False if skip_par else par_prepared.intersects(track_line)

    intersect_results.append({
        "SID": sid,
        "intersects_ph": intersects_ph,
        "intersects_par": intersects_par
    })

# Wrap in DataFrame
df_intersects = pd.DataFrame(intersect_results)

# Final counts
n_track_in_ph = df_intersects["intersects_ph"].sum()
n_track_in_par = df_intersects["intersects_par"].sum()

print(f"{n_track_in_ph} TCs intersected PH buffer.")
print(f"{n_track_in_par} TCs intersected PAR.")


461 TCs intersected PH buffer.
957 TCs intersected PAR.


In [33]:
# Step 1: Get SIDs that intersect PH and PAR
sids_ph = df_intersects[df_intersects["intersects_ph"]]["SID"]
sids_par = df_intersects[df_intersects["intersects_par"]]["SID"]

# Step 2: Count number of 3-hourly points within each region

# For PH
ph_lifetime = (
    gdf_tracks[gdf_tracks["SID"].isin(sids_ph)]
    .groupby("SID")
    .size()
    .reset_index(name="n_points")
)
ph_lifetime["hours_total_ph"] = ph_lifetime["n_points"] * 3

# For PAR
par_lifetime = (
    gdf_tracks[gdf_tracks["SID"].isin(sids_par)]
    .groupby("SID")
    .size()
    .reset_index(name="n_points")
)
par_lifetime["hours_total_par"] = par_lifetime["n_points"] * 3

# Step 3: Compute average
avg_total_ph_hours = ph_lifetime["hours_total_ph"].mean()
avg_total_par_hours = par_lifetime["hours_total_par"].mean()

print(f"Average lifetime of TCs intersecting Ph (for {len(ph_lifetime)} storms): {avg_total_ph_hours:.1f} hours")
print(f"Average lifetime of TCs intersecting PAR (for {len(par_lifetime)} storms): {avg_total_par_hours:.1f} hours")


Average lifetime of TCs intersecting Ph (for 461 storms): 215.6 hours
Average lifetime of TCs intersecting PAR (for 957 storms): 225.7 hours


In [28]:
# Filter points inside PH buffer
ph_inside = gdf_tracks[
    (gdf_tracks["SID"].isin(sids_ph)) & 
    (gdf_tracks.geometry.within(ph_buffered))
]

# Count points per SID
ph_lifetime = (
    ph_inside.groupby("SID")
    .size()
    .reset_index(name="n_points")
)
ph_lifetime["hours_inside_ph"] = ph_lifetime["n_points"] * 3

# Compute average lifetime inside PH
avg_ph_hours = ph_lifetime["hours_inside_ph"].mean()
print(f"Average TC lifetime *inside PH buffer*: {avg_ph_hours:.1f} hours (based on {len(ph_lifetime)} storms)")


Average TC lifetime *inside PH buffer*: 27.8 hours (based on 460 storms)


In [29]:
# Filter points inside PAR
par_inside = gdf_tracks[
    (gdf_tracks["SID"].isin(sids_par)) & 
    (gdf_tracks.geometry.within(par_poly))
]

# Count points per SID
par_lifetime = (
    par_inside.groupby("SID")
    .size()
    .reset_index(name="n_points")
)
par_lifetime["hours_inside_par"] = par_lifetime["n_points"] * 3

# Compute average lifetime inside PAR
avg_par_hours = par_lifetime["hours_inside_par"].mean()
print(f"Average TC lifetime *inside PAR*: {avg_par_hours:.1f} hours (based on {len(par_lifetime)} storms)")


Average TC lifetime *inside PAR*: 85.4 hours (based on 946 storms)
